# Lorenz 63 problem setup and algorithm testing
By: Rebecca Gjini 


In [ ]:
#Import statements
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numba
from numba import jit, njit
from scipy.linalg import sqrtm
from scipy.optimize import least_squares
import main.EnsembleKalmanAlgorithms as EKA
import main.l63.L63_model as l63
from cycler import cycler

#Plot preferences from palettable.colorbrewer.qualitative import Set1_9
from palettable.colorbrewer.qualitative import Dark2_8

#Plot Preferences 
plt.rcParams['figure.figsize'] = [11.0, 9.0] #size (w, h)
plt.rcParams['figure.dpi'] = 80
plt.rcParams['savefig.dpi'] = 400

plt.rcParams['font.size'] = 18 # controls default text sizes
plt.rcParams['legend.fontsize'] = 'large' # legend fontsize
plt.rcParams['figure.titlesize'] = 'large' # fontsize of the figure title  
plt.rcParams['axes.titlesize'] = 18     # fontsize of the axes title
plt.rcParams['axes.labelsize'] = 32   # fontsize of the x and y labels 35
plt.rcParams['xtick.labelsize'] = 20    # fontsize of the tick labels 17
plt.rcParams['ytick.labelsize'] = 20   # fontsize of the tick labels 17
plt.rcParams['axes.spines.right'] = False #makes right line in plot disappear
plt.rcParams['axes.spines.top'] = False #makes top line in plot disappear
plt.rcParams["font.family"] = "Times New Roman"

plt.rcParams['axes.prop_cycle'] = cycler(color=Dark2_8.mpl_colors)

In [ ]:
file_path = "main/l63/ces/data/"

#initialize random seed
np.random.seed(11)

#Creating my sythetic data

#initialize constants
og_sigma = 10.0
og_rho = 28.0
og_beta = 8.0 / 3.0
og_theta = np.array([og_sigma, og_rho, og_beta]) #we want to only find the last two

#time step
time_step = 0.01
#total time
T = 40
T_long = 390
#initial state for spinup
setup_x0 = np.array([1.0, 1.0, 1.0])

In [ ]:
#Find the initial condition for my data
spin_up_array = l63.runge_kutta_v(og_theta, setup_x0, time_step, T_long)
#intital condition used for the data
x0 = spin_up_array[-1]
# np.savetxt(file_path + 'x0.txt', x0, delimiter = ',') 
print(x0)

In [ ]:
#plot Spin up to see attractor
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot(spin_up_array[-1000:, 0], spin_up_array[-1000:, 1], spin_up_array[-1000:, 2])
plt.show()

In [ ]:
ic_cov_sqrt = (x0*0.05)*np.identity(3) #make these vectors
# np.savetxt(file_path + 'ic_cov_sqrt.txt', ic_cov_sqrt, delimiter = ',') 

#output from using our original initial conditions and function parameters
model_out_y = l63.G([np.log(og_rho), np.log(og_beta)], og_sigma, x0, np.zeros((3,3)), time_step, T)

#Prior mean \mu
mu = np.array([3.3, 1.2])

#Data covariance
RKgam = l63.runge_kutta_v(og_theta, x0, time_step, T_long)[int(30/time_step):] 
gam = np.zeros((9,int(T_long/10) - 3))
for i in range(0, T_long - 30, 10):
    ii = int(i/10)
    gam[0:3,ii] = np.mean(RKgam[int(i/time_step):int((i+10)/time_step)], axis = 0)
    RK_cov_gam = np.cov(RKgam[int(i/time_step):int((i+10)/time_step)].T)
    gam[3:6,ii] = np.diag(RK_cov_gam)
    gam[6:8,ii] = RK_cov_gam[0,1:]
    gam[8,ii] =  RK_cov_gam[1,2]

R = np.cov(gam)

#Covariance of prior B
B = np.array([[0.15**2, 0],[0,0.5**2]])
[u_b, l_b, q_b] = np.linalg.svd(B)  #solve for sqrt and sqrt_in
B_sqrt = (u_b*np.sqrt(l_b))@u_b.T
B_sqrt_in = EKA.matrix_inv_sqrt(B)

[u_r, l_r, q_r] = np.linalg.svd(R)  #solve for sqrt and sqrt_in
R_sqrt_in = (u_r/np.sqrt(l_r))@u_r.T
R_sqrt = (u_r*np.sqrt(l_r))@u_r.T

#Define Identity Matrix
I = np.identity(11)

y = model_out_y #+ R_sqrt@np.random.normal(0, 1, size = (len(model_out_y)))  #Data
print(y)
print(R_sqrt_in@y)


#Save problem data, prior, and covarainces 
# np.savetxt(file_path + 'y.txt', y, delimiter = ',') 
# np.savetxt(file_path + 'R.txt', R, delimiter = ',')
# np.savetxt(file_path + 'mu.txt', mu, delimiter = ',')
# np.savetxt(file_path + 'B.txt', B, delimiter = ',')

In [ ]:
#Plot cost functions
def loss(rhos, betas, sigs, x0s, ys,
                  ic_cov_sqrts, time_steps, Ts, R_sqrt_ins, mus, B_sqrt_ins):
    data_model = 0.5*np.linalg.norm(R_sqrt_ins@(ys - l63.G(np.array([np.log(rhos), np.log(betas)]) 
                                                               ,sigs, x0s, ic_cov_sqrts,
                                                               time_steps, Ts)))**2
    prior = 0.5*np.linalg.norm(B_sqrt_ins@(mus - np.array([np.log(rhos), np.log(betas)])))**2
    return data_model + prior

In [ ]:
#Evaluating Cost Function for Rho: 
rho_vals = np.arange(-0.6, 0.5, 0.005) + og_rho

cost_function = np.zeros(len(rho_vals))
for i in range(0, len(rho_vals)): 
    cost_function[i] = loss(rho_vals[i], og_beta, og_sigma, x0, model_out_y,
                  0*ic_cov_sqrt, time_step, T, R_sqrt_in, mu, B_sqrt_in)
    
og_cost = loss(og_rho, og_beta, og_sigma, x0, y,
                  0*ic_cov_sqrt, time_step, T, R_sqrt_in, mu, B_sqrt_in)

plt.figure()
plt.plot(rho_vals, cost_function, linewidth = 3)
plt.scatter(og_rho, og_cost, c = 'blue')
plt.axvline(x = 28, linewidth = 3, linestyle = '--', c = 'dimgrey')
#plt.title('Cost function of rho')
plt.ylabel('Loss')
plt.xlabel('$\\rho$')
plt.show()
#Evaluating Cost Function for Beta: 
beta_vals = np.arange(-1.2, 0.72, 0.01) + og_beta

cost_function = np.zeros(len(beta_vals))
for i in range(0, len(beta_vals)): 
    cost_function[i] = loss(og_rho, beta_vals[i], og_sigma, x0, y,
                  0*ic_cov_sqrt, time_step, T, R_sqrt_in, mu, B_sqrt_in)


plt.figure()
plt.plot(beta_vals, cost_function, linewidth = 3)
plt.scatter(og_beta, og_cost, c = 'blue')
plt.axvline(x = 8.0/3.0, linewidth = 3, linestyle = '--', c = 'dimgrey')
#plt.title('Cost function of beta')
plt.ylabel('Loss')
plt.xlabel('$\\beta$')
plt.show()

In [ ]:
#Intitializing EKI ensemble

K = 10       #number of ensemble members

max_runs = 50   #set a maximum number of runs 

N_t = 2         #we only estimate beta and rho

a = 1           #adjustment factor
alpha = 1     #adjustment factor

u = np.zeros((N_t, K))    #initialize parameter ensemble
u[0,:] = np.random.normal(0, 1, size = K)
u[1,:] = np.random.normal(0, 1, size = K)
print(u)


In [ ]:
#TEKI Test 
teki_u, teki_f, exit = EKA.TEKI(l63.G, u, (og_sigma, x0, ic_cov_sqrt, time_step, T), 
                          y, R, mu, B, min_rmse = 1, method = 'rmse', 
                             tol_x = 0.001, tol_f = 0.001, max_iter = max_runs)
print(teki_f)
print(exit)
print(np.exp(teki_u))
f = l63.G(np.mean(teki_u, axis = 1), og_sigma, x0, ic_cov_sqrt, time_step, T)
print(np.linalg.norm(R_sqrt_in@(y - f))/np.sqrt(len(y)))

In [ ]:
#UKI Test
uki_u, uki_f, _ = EKA.UKI(l63.G, (og_sigma, x0, ic_cov_sqrt, time_step, T), 
                         y, R, mu, B, a, min_rmse = 1,  method = 'rmse', 
                       tol_x = 0.001, tol_f = 0.001, max_iter = max_runs)
print(uki_f)
f = l63.G(np.mean(uki_u, axis = 1), og_sigma, x0, ic_cov_sqrt, time_step, T)
print(np.linalg.norm(R_sqrt_in@(y - f))/np.sqrt(len(y)))

In [ ]:
#ETKI Test 
etki_u, etki_f, exit = EKA.ETKI(l63.G, u, (og_sigma, x0, ic_cov_sqrt, time_step, T), 
                          y, R, mu, B, min_rmse = 1, method = 'rmse', 
                          tol_x = 0.001, tol_f = 0.001, max_iter = max_runs)
print(etki_f)
print(exit)
f = l63.G(np.mean(etki_u, axis = 1), og_sigma, x0, ic_cov_sqrt, time_step, T)
print(np.linalg.norm(R_sqrt_in@(y - f))/np.sqrt(len(y)))

In [ ]:
#IEKF Test 
iekf_u, iekf_f, exit = EKA.IEKF(l63.G, u, (og_sigma, x0, ic_cov_sqrt, time_step, T), 
                          y, R, mu, B, alpha, min_rmse = 1, 
                           method = 'rmse', tol_x = 0.001, tol_f = 0.001, max_iter = max_runs)
print(iekf_f)
print(exit)
f = l63.G(np.mean(iekf_u, axis = 1), og_sigma, x0, ic_cov_sqrt, time_step, T)
print(np.linalg.norm(R_sqrt_in@(y - f))/np.sqrt(len(y)))

In [ ]:
#GN_CO Test 
gn_u, gn_f, exit = EKA.GN_CO(l63.G, u, (og_sigma, x0, ic_cov_sqrt, time_step, T), 
                          y, R, mu, B, min_rmse = 1, 
                           method = 'rmse', tol_x = 0.001, tol_f = 0.001, max_iter = max_runs)
print(gn_f)
print(exit)
f = l63.G(np.mean(gn_u, axis = 1), og_sigma, x0, ic_cov_sqrt, time_step, T)
print(np.linalg.norm(R_sqrt_in@(y - f))/np.sqrt(len(y)))

In [ ]:
solution = least_squares(l63.r, u[:,6], args=(og_sigma, x0, ic_cov_sqrt, time_step, 
                                          T, y, R_sqrt_in, mu, B_sqrt), method = 'lm', 
                         xtol = 1e-8, ftol=1e-8) # set diff_step = 1 for LM to do well
print(solution.nfev)
print(np.exp(B_sqrt@u[:,7] + mu))
print(np.exp(B_sqrt@solution.x + mu))
print(solution.status)

In [ ]:
label_list = ['UKI', 'TEKI', 'ETKI', 'IEKF', 'Finite Diff']
function_evals = [uki_f, teki_f, etki_f, iekf_f, solution.nfev]

# creating the bar plot
plt.figure()
plt.bar(label_list, function_evals, color ='#6D61B1', 
        width = 0.4)
 
plt.xlabel("EKI Algorithms")
plt.ylabel("Number of forward function evals")
plt.title("EKI Showdown")
plt.show()